# Creating Benchmark Datasets and Defining the Model

## Data Genaration + Pre-Processing

In [1]:
from sklearn.datasets import make_classification
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error as MSE
from scipy.special import softmax

import numpy as np

from datetime import datetime as time

from pimp import permutation_importance

In [2]:
n_samples = 10000

In [3]:
scaler = MinMaxScaler()

Creating a dataset to evaluate the best parameters for the RandomForest model

In [4]:
eval_datasets = []
for noise_std in [0, 2, 4]:
    for informative_level in range(20, 101, 20): #in percents
        for n_features in range(20, 101, 40):
            X, y = make_classification(n_samples=n_samples,
                                       n_features=n_features,
                                       n_informative=int(informative_level * n_features / 100),
                                       n_redundant=0)
            # creating a noise with the same dimension as the dataset
            noise = np.random.normal(0, noise_std, [n_samples, n_features])
            X += noise
            # normalising the input data to the range [0,1]
            X = scaler.fit_transform(X)
            eval_datasets.append((X, y))

Creating a dataset to be used for the purposes of estimating the importance of features

In [5]:
X_train = []
y_train = []
informativeness = []
for noise_std in [0, 4]:
    for informative_level in [20]: #in percents
        n_features = 60
        n_informative = int(informative_level * n_features / 100)
        
        uniform_informativeness = 1 / n_informative
        curr_informativeness = []
        for feature in range(n_features):
            if feature < n_informative:
                curr_informativeness.append(uniform_informativeness)
            else:
                curr_informativeness.append(0)
        informativeness.append(curr_informativeness)
        
        X, y = make_classification(n_samples=n_samples,
                                    n_features=n_features,
                                    n_informative=n_informative,
                                    n_redundant=0)
        # creating a noise with the same dimension as the dataset
        noise = np.random.normal(0, noise_std, [n_samples, n_features])
        X += noise
        # normalising the input data to the range [0,1]
        X = scaler.fit_transform(X)
        X_train.append(X)
        y_train.append(y)

true_importance = np.mean(np.array(informativeness), axis=0)
            
X_train = np.vstack(X_train)
y_train = np.hstack(y_train)

## Model Optimization - Random Search for Classification

In [ ]:
#define a model
model = RandomForestClassifier()

In [ ]:
# define evaluation
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=1)

In [ ]:
# define search space
space = dict()
space['n_estimators'] = list(range(500, 1001, 200))
space['max_depth'] = list(range(3, 8))
space['min_samples_split'] = list(range(1, 4))

In [ ]:
# define search
search = RandomizedSearchCV(model, space, n_iter=45, scoring='accuracy', n_jobs=-1, cv=cv, random_state=1, verbose=10)

In [ ]:
# execute search
result = search.fit(X_train, y_train)

Fitting 10 folds for each of 45 candidates, totalling 450 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  9.0min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed: 13.0min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed: 17.7min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed: 22.8min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed: 22.9min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed: 23.6min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed: 31.2min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 40.6min
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed: 50

In [ ]:
# summarize result
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

Best Score: 0.6869375000000001
Best Hyperparameters: {'n_estimators': 700, 'min_samples_split': 2, 'max_depth': 7}


## Final Model

In [6]:
final_model = RandomForestClassifier(n_estimators=700,
                                     max_depth=7,
                                     min_samples_split=2,
                                     bootstrap=True)

start = time.now()
final_model.fit(X_train, y_train)
end = time.now()
print(f'Finished fitting model in {(end - start)}')

Finished fitting model in 0:00:53.395958


Performing the permutation importance using the model we trained on the simualted data

In [7]:
start = time.now()
scores = permutation_importance(final_model, X_train, y_train)
end = time.now()
print(f'Finished permutation importance in {(end - start)}')

for key, value in scores.items():
    print(f'{key}:')
    print(value)
    print()

Finished permutation importance in 0:06:34.030857
importances:
[[-5.0000e-05  6.5000e-04  7.0000e-04  2.5000e-04  4.0000e-04]
 [ 1.5700e-02  1.4000e-02  1.4300e-02  1.3100e-02  1.3150e-02]
 [ 2.2150e-02  2.0100e-02  2.1750e-02  2.2200e-02  2.1450e-02]
 [ 2.5350e-02  2.5900e-02  2.4550e-02  2.6150e-02  2.4250e-02]
 [ 2.4900e-02  2.5250e-02  2.5350e-02  2.5500e-02  2.5250e-02]
 [ 2.8000e-02  2.8000e-02  2.8400e-02  2.8200e-02  2.8200e-02]
 [ 3.4500e-02  3.3250e-02  3.3400e-02  3.3000e-02  3.2300e-02]
 [ 3.3750e-02  3.3500e-02  3.2900e-02  3.3500e-02  3.3400e-02]
 [ 3.4050e-02  3.4250e-02  3.4150e-02  3.3800e-02  3.3800e-02]
 [ 3.4450e-02  3.4600e-02  3.3850e-02  3.4500e-02  3.4600e-02]
 [ 3.5300e-02  3.4700e-02  3.5350e-02  3.4950e-02  3.4800e-02]
 [ 4.9700e-02  4.9400e-02  5.0700e-02  5.1000e-02  5.2750e-02]
 [ 5.3650e-02  5.3800e-02  5.3800e-02  5.3400e-02  5.3500e-02]
 [ 5.3850e-02  5.4300e-02  5.4300e-02  5.4150e-02  5.4000e-02]
 [ 5.5650e-02  5.5550e-02  5.5250e-02  5.5700e-02  5.58

Using softmax to normalize the permutation importance scores to sum up to 1.

In [8]:
permutation_scores = {}
permutation_scores['mean'] = softmax(scores['mean'])
permutation_scores['p_vals'] = softmax(1 - scores['p_vals'])

Defining the fusion model

In [9]:

def fusion_scores(*lists, weights=None):
	"""
	Fusion scores of given lists of scores.

	Parameters
	----------
	*lists : list , shape (n_features)
	    Scores lists to be fused.
	weights : list , shape (n_lists), default=None
	    Weights assigned to each list
	    
	Returns
	-------
	result : list
	    a fused list

	Reference
	---------
	C. C. Vogt and G. W. Cottrell:
	“Fusion via linear combination of scores.”
	Information Retrieval, 1, 151–173 (1999) Kluwer Academic Publishers.
	Manufactured in The Netherlands.
	http://cseweb.ucsd.edu/~gary/pubs/info-retrieval-1999.pdf
	"""

	if not weights:
		weights = np.ones(len(lists))
	else:
		weights = np.array(weights)
	weights = np.expand_dims(weights, axis=1)

	fusion_scores = np.sum((np.multiply(*lists, weights)), axis=0)
	return fusion_scores

Evaluating the performance of the various models given the ground truth we have from the data generation process

In [10]:
def eval(ground_truth, fusion_scores):
    return MSE(ground_truth, fusion_scores, squared=False)

f_scores = fusion_scores(list(permutation_scores.values()), weights=[0.5, 0.5])

models_scores = {
                'classic PIMP': permutation_scores['mean'], 
                'p-value PIMP': permutation_scores['p_vals'],
                'fusion PIMP': f_scores
                 }

for model_type, model_importances in models_scores.items():
    MSE_scores = eval(true_importance, model_importances)
    print(f'{model_type} MSE: {MSE_scores}\n')


classic PIMP MSE: 0.03444363350305197

p-value PIMP MSE: 0.0368190447419121

fusion PIMP MSE: 0.03561195002966417

